In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        getOrCreate()

22/01/23 20:25:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.option("header", "true").csv("data/*.gz")


In [3]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F

df = df.withColumn("friends_count",col("friends_count").cast(IntegerType()))
df3 = df
df4 = df
df = df.select("screen_name","user_id","friends_count","verified").distinct().orderBy(col("friends_count").desc())
df = df.where(col("verified") == "TRUE")
df = df.where(col("friends_count").isNotNull())
df = df.where(col("user_id").isNotNull())
df2 = df
df = df.select("*").groupBy("user_id").agg(F.max("friends_count").alias("max")).orderBy(col("max").desc())

df2 = df2.select("*").groupBy("user_id").agg(F.min("friends_count").alias("min")).orderBy(col("min").desc())

In [4]:
df = df.join(df2, ['user_id'])


In [5]:
df = df.withColumn("diff", (df['max'] - df['min']))
df = df.orderBy(col("diff").desc())
df = df.limit(1000)

In [6]:
df3 = df3.where(col("user_id").isNotNull())
df3 = df3.groupBy("user_id").count().orderBy(col("count").desc())
df3 = df3.orderBy(col("count").desc())
df3 = df3.limit(1000)
#df = df.join(df3, df.user_id == df3.user_id, 'inner')
df = df.join(df3, ['user_id'])

In [11]:
df.show()

+---------+------+------+----+-----+
|  user_id|   max|   min|diff|count|
+---------+------+------+----+-----+
|367210075| 79069| 75195|3874|  614|
| 81081065| 10382|  7450|2932| 1912|
|201079922| 11900|  9226|2674|  672|
|      000|  1632|    64|1568| 5964|
| 91946403| 95700| 94628|1072|  975|
| 14562685| 95523| 94592| 931| 1280|
|117109734|  2843|  2097| 746|  600|
| 18936284|108490|107896| 594| 1135|
| 67690947| 10898| 10356| 542|  771|
| 14434407|   540|   100| 440|  650|
| 22703339|  1407|   976| 431| 1214|
| 74421223|  4769|  4396| 373|  756|
| 14307494|  6936|  6645| 291|  582|
| 18831926|  3888|  3645| 243|  924|
| 58048133|  1997|  1769| 228|  657|
| 34779125|  4377|  4183| 194|  564|
|365216346|  4689|  4539| 150|  664|
+---------+------+------+----+-----+



In [10]:
from pyspark.ml.feature import NGram
from pyspark.sql.types import StringType

df4 = df4.where(col("text").isNotNull())
df4 = df4.withColumn("text",col("text").cast(StringType()))

df_collect = df.collect()

for row in df_collect:
    df_ngram = df4.where(col("user_id") == row['user_id'])
    
    df_new = df_ngram.withColumn("testing", split(col("text")," "))
    ngram = NGram().setN(3).setInputCol("testing").setOutputCol("ngrams")
    ngramDataFrame = ngram.transform(df_new)
    print(row['user_id'])
    ngramDataFrame.select("ngrams").show()

367210075


+--------------------+
|              ngrams|
+--------------------+
|[#LoMásLeído | El...|
|[#LoMásLeído | Re...|
|[#LoMásLeído | @C...|
|[#EspecialCoronav...|
|[#EspecialCoronav...|
|[#EspecialCoronav...|
|[#EspecialCoronav...|
|[🔴#ÚltimaHora Es...|
|[#EspecialCoronav...|
|[#LupaSanitaria ?...|
|[#Madrid | Los, |...|
|[#HéroesDeLaSalud...|
|[#Cantabria | @ca...|
|[#EspecialCoronav...|
|[#EspecialCoronav...|
|[#EspecialCoronav...|
|[#TweetSaludable ...|
|[#EspecialCoronav...|
|[#Exclusiva | 25,...|
|[#EspecialCoronav...|
+--------------------+
only showing top 20 rows

81081065


+--------------------+
|              ngrams|
+--------------------+
|["""No tienen din...|
|[Padres y profeso...|
|["Las cuidadoras ...|
|[El alcalde de, a...|
|[👏 Innumerables ...|
|[¿Cuáles han sido...|
|["Grandes propiet...|
|[👉 [ENCUESTA] ¿E...|
|[#Manlleu trabaja...|
|[Los líderes inde...|
|["Los grandes ten...|
|["""Miles de caso...|
|["""Nos mandan de...|
|[Hoy la firma, la...|
|[💪 Así trabajan,...|
|[En estos días, e...|
|[🔴 #ÚltimaHora ?...|
|[Las cárceles cat...|
|[👉 📲 [ENCUESTA,...|
|["Los grandes pro...|
+--------------------+
only showing top 20 rows

201079922


+--------------------+
|              ngrams|
+--------------------+
|[Confirma goberna...|
|[#Coronavirus en ...|
|[Lee @m_ebrard de...|
|[Reporte #coronav...|
|[Casos confirmado...|
|[Reporte #Coronav...|
|[El Empire State,...|
|[Mujer de 95, de ...|
|[Casos de #Corona...|
|[Dudan en Wuhan, ...|
|[Muere niña de, n...|
|[En Khurramshahr,...|
|[Guardan minuto d...|
|[Informa @HectorA...|
|[Trasciende que d...|
|[En conferencia d...|
|[En China mantien...|
|[Llegan cóndores ...|
|[Chris Cuomo, her...|
|[#Coronavirus en ...|
+--------------------+
only showing top 20 rows

000


+--------------------+
|              ngrams|
+--------------------+
|[000 people infec...|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|[ I mean, I mean ...|
|   [Twitter Web App]|
|[000 de veces, de...|
|[000 pacientes se...|
|[000 de veces, de...|
|   [Twitter Web App]|
|[000 to provide, ...|
|                  []|
|[000 patients are...|
|                  []|
|                  []|
|                  []|
|[038 recoveries i...|
|         [000 / 100]|
+--------------------+
only showing top 20 rows

91946403


...

+--------------------+
|              ngrams|
+--------------------+
|[The latest insta...|
|["""P"" stands fo...|
|[I'll use this, u...|
|[So empathize. Th...|
|[Where do you, do...|
|["""E"" stands fo...|
|["""C"" stands fo...|
|["""R"" stands fo...|
|[Before ringing t...|
|[Track details of...|
|["""S"" stands fo...|
|[These are trying...|
|[How many times, ...|
|[(c) General mana...|
|["(b) When they, ...|
|[I’m generally mo...|
|[My morning cry, ...|
|[@EFMoriarty Than...|
|[I’ve tried to, t...|
|[The care here, c...|
+--------------------+
only showing top 20 rows

18831926


+--------------------+
|              ngrams|
+--------------------+
|[This case count,...|
|[New York death, ...|
|[Should we all, w...|
|[📍Illinois: ‘We ...|
|[⚠️MORTALITY REPO...|
|[@Plan_Prep_Live ...|
|[⚠️MASK please!!!...|
|[Huge 20x underco...|
|[2) “residents sa...|
|["3) “It can't, “...|
|[4) “https://t.co...|
|[“Funeral homes h...|
|[6) “Another popu...|
|[7) “A source, “A...|
|[8) “The source, ...|
|["9) ""There have...|
|[10) “nobody in, ...|
|["11) ""But durin...|
|["12) ""Anyone lo...|
|[13) BOTTOMLINE: ...|
+--------------------+
only showing top 20 rows

58048133


+--------------------+
|              ngrams|
+--------------------+
|[Médicos europeos...|
|[Revisa cuáles so...|
|[Sin bajarse: Red...|
|[Higiene y salud:...|
|[Juez de Texas, d...|
|[Balance eleva a,...|
|[Instituto de Sal...|
|[🔴 Subsecretario...|
|[🔴 AMPLIADO Mins...|
|[🔴 Subsecretaria...|
|[Plácido Domingo ...|
|[🔴 LO ÚLTIMO, LO...|
|["FDA autoriza en...|
|[¿Síntomas leves ...|
|[Cuatro miembros ...|
|["🔴  Ministro,  ...|
|[🔴 Ministro de, ...|
|[🔴 Francia regis...|
|[Estados Unidos s...|
|[Actor de Star, d...|
+--------------------+
only showing top 20 rows

34779125


+--------------------+
|              ngrams|
+--------------------+
|["No #RodaViva, @...|
|[Íntegra da notíc...|
|[#Covid19: Epidem...|
|[#UtilidadePúblic...|
|[Cara de pau, de ...|
|[#UtilidadePúblic...|
|[🇺🇸🚨 Iluminaçã...|
|[No #RodaViva, pe...|
|[#RendaBásica, an...|
|[DOU de hoje:, de...|
|[📈💻 Amanhã (2/a...|
|[🇧🇷 Gramado do,...|
|[Capitais brasile...|
|[A maior tragédia...|
|[Sargento de 46, ...|
|[Necropolítica de...|
|[OMS responde a, ...|
|[Atenção: Em cade...|
|[#UtilidadePúblic...|
|[O número de, núm...|
+--------------------+
only showing top 20 rows

365216346


+--------------------+
|              ngrams|
+--------------------+
|[Latest on #COVID...|
|[The US added, US...|
|[China added 48, ...|
|[Argentina added ...|
|[China has postpo...|
|[Latest on #COVID...|
|[Taiwan added 16,...|
|[Latest on #COVID...|
|[The US added, US...|
|[Spain added 6461...|
|[Germany added 12...|
|[Iran reported an...|
|[The UK added, UK...|
|[Switzerland adde...|
|[Belgium added 87...|
|[The Netherlands ...|
|[Austria added 42...|
|[South Korea adde...|
|[Canada added 26,...|
|[Portugal added 1...|
+--------------------+
only showing top 20 rows

